# Lesson 3: Reflection and Blogpost Writing
## Setup

In [1]:
import os

from dotenv import load_dotenv

load_dotenv()


llm_config = {
    "model": "gpt-4o-mini",
    "api_key": os.getenv("OPENAI_API_KEY"),
}

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [4]:
print(reply)

**Unlocking the Future with DeepLearning.AI**

DeepLearning.AI is revolutionizing the landscape of artificial intelligence education. Founded by Andrew Ng, it offers a suite of online courses designed to empower aspiring AI practitioners and seasoned professionals alike. From foundational concepts in machine learning to specialized topics in deep learning, the curriculum is accessible and engaging. With a hands-on approach, learners gain practical skills that translate to real-world applications. Additionally, the vibrant community fosters collaboration and innovation, making it an invaluable resource for anyone looking to deepen their understanding of AI. Dive into the future—enhance your career with DeepLearning.AI.


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [5]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

**Unleashing the Power of AI with DeepLearning.AI**

DeepLearning.AI is transforming the landscape of artificial intelligence education and innovation. Founded by AI pioneer Andrew Ng, it offers comprehensive online courses that empower learners to master deep learning and neural networks. With a practical, hands-on approach, students engage with real-world projects and cutting-edge technology. The platform also fosters a vibrant community, connecting enthusiasts and experts alike. Whether you’re a beginner or looking to enhance your skills, DeepLearning.AI equips you with the knowledge needed to thrive in the AI revolution, making complex concepts accessible and exciting for all. Join the movement today!

------------------------

## Nested chat

In [10]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO-Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

legal_reviewer = autogen.AssistantAgent(
    name="Legal-Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

ethics_reviewer = autogen.AssistantAgent(
    name="Ethics-Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

meta_reviewer = autogen.AssistantAgent(
    name="Meta-Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [11]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [12]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

In [13]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

**Unleashing the Power of AI with DeepLearning.AI**

DeepLearning.AI is transforming the landscape of artificial intelligence education and innovation. Founded by AI pioneer Andrew Ng, it offers comprehensive online courses that empower learners to master deep learning and neural networks. With a practical, hands-on approach, students engage with real-world projects and cutting-edge technology. The platform also fosters a vibrant community, connecting enthusiasts and experts alike. Whether you’re a beginner or looking to enhance your skills, DeepLearning.AI equips you with the knowledge needed to thrive in the AI revolution, making complex concepts accessible and exciting for all. Join the movement today!

------------------------

In [14]:
print(res.summary)

**Elevate Your Skills with DeepLearning.AI: The Leading Hub for AI Education**

DeepLearning.AI is revolutionizing the world of AI education by offering top-tier online courses for aspiring data scientists and machine learning enthusiasts. Founded by the renowned Andrew Ng, this platform provides deep learning certifications and comprehensive resources, making advanced AI concepts accessible to everyone. 

With a hands-on approach, learners tackle real-world projects and scenarios, connecting theory with practice. DeepLearning.AI’s vibrant community fosters collaboration and support, further enhancing the learning experience. By joining today, you’ll not only boost your career prospects in the booming AI field but also gain the confidence to tackle complex challenges effectively. Explore the future of machine learning education!
